# Day 5 - Supply Stacks

https://adventofcode.com/2022/day/5

In [189]:
from pathlib import Path
from copy import deepcopy

INPUTS = Path('input.txt').read_text().strip()

In [190]:
stacks, instructions = INPUTS.split('\n\n')
instructions_split = instructions.split('\n')


The stacks take the form of what they best look like visually, but programmatically it's better to do a few things:
- Reduce the items to their individual characters in a list.
- Transpose those items to they sit in rows.

This allows use to do popping, sorting, and appending to those row lists, rather than mess with the visual columns.

Starting off we'll reverse the stack and split it up, so our stack numbers become headers and the stacks grow downward.

In [191]:
stacks_reversed = stacks.split('\n')[::-1]
full_width = len(stacks_reversed[0]) + 1
stacks_reversed

[' 1   2   3   4   5   6   7   8   9',
 '[S] [B] [B] [F] [H] [C] [B] [N] [L]',
 '[M] [W] [J] [R] [V] [B] [J] [C] [S]',
 '[R] [D] [F] [P] [R] [P] [R] [S] [G]',
 '[N] [J] [H] [B] [P] [T] [P] [L]',
 '[W] [Q] [D] [M] [T]     [L] [T]',
 '[J] [P] [R] [N] [B]         [Z]',
 '[V] [C] [P] [D]             [B]',
 '[T] [V]                     [W]']

Next we adjust the stacks slightly by injecting "blank" values that aren't spaces. These will be `-` characters.

Within the original row strings, we do this by replacing four (4) empty spaces with ` [-]`, which fills in the spacer and the dummy value. We also need to tack on some extra spaces for dummy values in the last column(s).

Finally, we can take those normalized strings (all the same width, all identical formats), strip out leading/trailing brackets, and actually *split* by those interleaving brackets (`] [`) to get the individual characters.

In [192]:
stacks_adjusted = []
for stack in stacks_reversed[1:]:
    new_stack = stack.ljust(full_width).replace('    ', ' [-]')
    new_stack = new_stack.lstrip('[').rstrip(']').split('] [')
    stacks_adjusted.append(new_stack)
stacks_adjusted

[['S', 'B', 'B', 'F', 'H', 'C', 'B', 'N', 'L'],
 ['M', 'W', 'J', 'R', 'V', 'B', 'J', 'C', 'S'],
 ['R', 'D', 'F', 'P', 'R', 'P', 'R', 'S', 'G'],
 ['N', 'J', 'H', 'B', 'P', 'T', 'P', 'L', '-'],
 ['W', 'Q', 'D', 'M', 'T', '-', 'L', 'T', '-'],
 ['J', 'P', 'R', 'N', 'B', '-', '-', 'Z', '-'],
 ['V', 'C', 'P', 'D', '-', '-', '-', 'B', '-'],
 ['T', 'V', '-', '-', '-', '-', '-', 'W', '-']]

Finally, the tranposition, a simple `list(zip(*list_of_lists))` does the trick. We adjust that to also filter out the now-trailing `-` values, so our final rows only contain the actual container items.

In [193]:
stacks_transposed = [[y for y in x if y != '-'] for x in zip(*stacks_adjusted)]
stacks_transposed

[['S', 'M', 'R', 'N', 'W', 'J', 'V', 'T'],
 ['B', 'W', 'D', 'J', 'Q', 'P', 'C', 'V'],
 ['B', 'J', 'F', 'H', 'D', 'R', 'P'],
 ['F', 'R', 'P', 'B', 'M', 'N', 'D'],
 ['H', 'V', 'R', 'P', 'T', 'B'],
 ['C', 'B', 'P', 'T'],
 ['B', 'J', 'R', 'P', 'L'],
 ['N', 'C', 'S', 'L', 'T', 'Z', 'B', 'W'],
 ['L', 'S', 'G']]

**Now** we can do the real work of following the instructions. Instructions take the form `move X from Y to Z`.

- `X` is the number of items to `.pop` from the end of a stack. Those items are reversed before being appended to the target stack.
- `Y` and `Z` are the stack column indices. The instructions are 1-based, so we just subtract one to determine the indices we need to `.pop` from and `.append` to.

In [194]:
import re

pattern = re.compile(r"move (\d{1,3}) from (\d{1,3}) to (\d{1,3})")


In [195]:
def move_stacks(
    stacks: list[list[str]],
    instructions: list[str],
    reverse: bool = True,
) -> list[list[str]]:
    stacks_moved = deepcopy(stacks)
    for instruction in instructions:
        if not (match := pattern.search(instruction)):
            continue
        num_, from_, to_ = map(int, match.groups())
        # Pick out out from- and to-stacks to manipulate
        from_stack, to_stack = stacks_moved[from_ - 1], stacks_moved[to_ - 1]
        # Split the from-stack into the remaining containers left on the stack
        # and the in-motion containers being picked off
        # (this is generally faster than repeatedly `.pop`ping them)
        from_stack, in_motion = from_stack[:-num_], from_stack[-num_:]
        # Place the in-motion containers at the end of the to-stack.
        # (they must be reversed when placed)
        if reverse:
            in_motion = in_motion[::-1]
        to_stack += in_motion
        # Re-assign the updated from- and to-stack lists to the original positions
        stacks_moved[from_ - 1] = from_stack
        stacks_moved[to_ - 1] = to_stack
    return stacks_moved


In [196]:
stacks_moved_part1 = move_stacks(
    stacks=stacks_transposed,
    instructions=instructions_split,
    reverse=True,
)


Now we just need the "code" for the containers on the tops of each stack.

In [197]:
result = "".join(x[-1] for x in stacks_moved_part1)
print(f"{result=}")

result='LJSVLTWQM'


## Part 2

This follows the same pattern as before, except we just don't need to reverse the in-motion containers when extending the to-stack. Simple enough.

The original code was as follows:

```py
stacks_moved = deepcopy(stacks_transposed)
for instruction in instructions_split:
    if not (match := pattern.search(instruction)):
        continue
    num_, from_, to_ = map(int, match.groups())
    # Pick out out from- and to-stacks to manipulate
    from_stack, to_stack = stacks_moved[from_ - 1], stacks_moved[to_ - 1]
    # Split the from-stack into the remaining containers left on the stack
    # and the in-motion containers being picked off
    # (this is generally faster than repeatedly `.pop`ping them)
    from_stack, in_motion = from_stack[:-num_], from_stack[-num_:]
    # Place the in-motion containers at the end of the to-stack.
    # (they must be reversed when placed)
    to_stack += in_motion[::-1]
    # Re-assign the updated from- and to-stack lists to the original positions
    stacks_moved[from_ - 1] = from_stack
    stacks_moved[to_ - 1] = to_stack
```

We're going to go back and refactor this to just wrap it in a function, `move_stacks`, and include a keyword for `reverse=True`, so part 1 and 2 can re-use the same function.

In [198]:
stacks_moved_part2 = move_stacks(
    stacks=stacks_transposed,
    instructions=instructions_split,
    reverse=False,
)


In [199]:
result = "".join(x[-1] for x in stacks_moved_part2)
print(f"{result=}")

result='BRQWDBBJM'
